In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


In [4]:
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [5]:
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [6]:
question = "지구 온난화의 심각성 대해 알려주세요."

parser = JsonOutputParser(pydantic_object=Topic)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser 

chain.invoke({"question": question})  


{'description': '지구 온난화는 지구의 평균 기온이 지속적으로 상승하는 현상으로, 이는 주로 인간 활동에 의해 발생하는 온실가스 배출로 인해 발생합니다. 이로 인해 극지방의 빙하가 녹고 해수면이 상승하며, 기후 변화로 인한 자연 재해가 빈번해지고 있습니다.',
 'hashtags': '#지구온난화 #기후변화 #온실가스'}

In [7]:
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

response = chain.invoke({"question": question})

print(response)

{'description': '지구 온난화는 대기 중 온실가스 농도의 증가로 인해 지구의 평균 기온이 상승하는 현상을 말합니다. 이는 기후 변화, 해수면 상승, 극지방 빙하의 감소 등 다양한 환경 문제를 초래합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#해수면상승', '#빙하감소']}
